## Re-clasificacion del train

In [1]:
import torch
import torch.nn as nn  # Importación necesaria para nn.Linear
import shutil
import os
from torchvision import transforms
from PIL import Image
from torchvision import models


In [2]:

# 1. Función para cargar el modelo entrenado
def cargar_modelo(ruta_modelo, device):
    model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 2)  # Ajustar para 2 clases (violencia/no violencia)
    
    # Cargar el modelo entrenado
    model.load_state_dict(torch.load(ruta_modelo, map_location=device))
    model.to(device)
    model.eval()  # Cambiar a modo de evaluación
    return model

# 2. Transformaciones simples: solo convertir a tensor
transform = transforms.Compose([
    transforms.ToTensor(),  # Convertir la imagen a tensor sin normalización
])
def clasificar_y_copiar_por_confianza(modelo, carpeta_origen, carpeta_destino, umbral=0.9055, device='cpu'):
    # Iterar sobre los archivos en la carpeta de origen
    for nombre_imagen in os.listdir(carpeta_origen):
        ruta_imagen = os.path.join(carpeta_origen, nombre_imagen)
        ruta_destino = os.path.join(carpeta_destino, nombre_imagen)

        # Verificar que es un archivo y no un directorio
        if os.path.isfile(ruta_imagen):
            try:
                # Cargar la imagen
                imagen = Image.open(ruta_imagen)
                imagen = transform(imagen).unsqueeze(0).to(device)  # Aplicar las transformaciones y agregar batch dimension

                # Clasificar la imagen
                with torch.no_grad():
                    output = modelo(imagen)
                    probabilidades = torch.softmax(output, dim=1)  # Convertir la salida a probabilidades
                    confianza, _ = torch.max(probabilidades, 1)  # Obtener la mayor probabilidad (la confianza)

                confianza = confianza.item()

                # Solo copiar la imagen si la confianza es mayor al umbral
                if confianza >= umbral:
                    # Crear el directorio destino si no existe
                    if not os.path.exists(carpeta_destino):
                        os.makedirs(carpeta_destino)

                    shutil.copy(ruta_imagen, ruta_destino)  # Copiar en lugar de mover
                    print(f">>>>   Imagen {nombre_imagen} copiada con confianza {confianza*100:.2f}% a {carpeta_destino}")
                else:
                    print(f"Imagen {nombre_imagen} no copiada por baja confianza ({confianza*100:.2f}%)")
            except Exception as e:
                print(f"Error al procesar la imagen {nombre_imagen}: {e}")

In [3]:
if __name__ == "__main__":
    # Ruta al modelo guardado
    ruta_modelo = "../SecondDatabase/clasificados/modelo/ResNet18Original.pth"

    # Directorios para las imágenes clasificadas (destino)
    directorio_violencia = "../SecondDatabase/clasificados/violencia/"
    directorio_no_violencia = "../SecondDatabase/clasificados/no_violencia/"

    # Carpetas del nuevo dataset que contienen imágenes para clasificar (origen)
    carpeta_violencia = "../SecondDatabase/violencia/"  # Ruta a la carpeta de imágenes de violencia
    carpeta_no_violencia = "../SecondDatabase/no_violencia/"  # Ruta a la carpeta de imágenes de no violencia

    # Cargar el modelo
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = cargar_modelo(ruta_modelo, device)

    # Clasificar las imágenes en la carpeta de violencia y copiarlas si cumplen con el umbral
    print(f"\nClasificando imágenes en la carpeta de violencia: {carpeta_violencia}")
    clasificar_y_copiar_por_confianza(model, carpeta_violencia, directorio_violencia, umbral=0.6905, device=device)

    # Clasificar las imágenes en la carpeta de no violencia y copiarlas si cumplen con el umbral
    #print(f"\nClasificando imágenes en la carpeta de no violencia: {carpeta_no_violencia}")
    #clasificar_y_copiar_por_confianza(model, carpeta_no_violencia, directorio_no_violencia, umbral=0.9904, device=device)

 


Clasificando imágenes en la carpeta de violencia: ../SecondDatabase/violencia/
>>>>   Imagen violencia_100684.jpg copiada con confianza 69.27% a ../SecondDatabase/clasificados/violencia/
Imagen violencia_100779.jpg no copiada por baja confianza (68.95%)
Imagen violencia_10134.jpg no copiada por baja confianza (69.01%)
>>>>   Imagen violencia_101463.jpg copiada con confianza 69.21% a ../SecondDatabase/clasificados/violencia/
>>>>   Imagen violencia_102460.jpg copiada con confianza 69.23% a ../SecondDatabase/clasificados/violencia/
>>>>   Imagen violencia_102500.jpg copiada con confianza 69.05% a ../SecondDatabase/clasificados/violencia/
Imagen violencia_102639.jpg no copiada por baja confianza (68.61%)
Imagen violencia_102789.jpg no copiada por baja confianza (68.77%)
>>>>   Imagen violencia_10283.jpg copiada con confianza 69.37% a ../SecondDatabase/clasificados/violencia/
Imagen violencia_105252.jpg no copiada por baja confianza (69.00%)
>>>>   Imagen violencia_105904.jpg copiada con 